In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from ase.io import read


In [23]:
def molfile_to_xyz(molfile:str):
    temp_file = 'temp2.mol'
    with open(temp_file, 'w+') as f:
        f.writelines(molfile) #write molfile to temporary file
    mol = Chem.MolFromMolFile(temp_file) # Read temp file into RDKit molecule
    print(type(mol))
    #mol = Chem.AddHs(mol)# post-process molecule
    AllChem.EmbedMolecule(mol)
    Chem.MolToMolFile(mol, temp_file)    # Write RDKit molecule to a temporary file
    molecule = read('temp.mol') # Read the temporary file into ASE Atoms object
    return molecule
FILE = 'temp.mol'
with open(FILE, 'r+') as f:
    content = f.readlines()
print(content)
print(molfile_to_xyz(content))

['\n', '     RDKit          3D\n', '\n', '  8  7  0  0  0  0  0  0  0  0999 V2000\n', '   -0.7533   -0.0093    0.0601 C   0  0  0  0  0  0  0  0  0  0  0  0\n', '    0.7504   -0.0263    0.0011 C   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.1589    0.1177    1.0753 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.0772    0.8427   -0.5977 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.1758   -0.9182   -0.4328 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.0733   -0.5749   -0.9198 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.2143   -0.4556    0.9130 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.1271    1.0239   -0.0993 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '  1  2  1  0\n', '  1  3  1  0\n', '  1  4  1  0\n', '  1  5  1  0\n', '  2  6  1  0\n', '  2  7  1  0\n', '  2  8  1  0\n', 'M  END\n']
<class 'rdkit.Chem.rdchem.Mol'>
Atoms(symbols='C2H6', pbc=False)


[13:25:53] Molecule does not have explicit Hs. Consider calling AddHs()


In [21]:
FILE = 'temp.mol'
with open(FILE, 'r+') as f:
    content = f.readlines()
print(content)

def molfile_to_xyz(molfile:str):
    temp_file = 'temp2.mol'
    with open(temp_file, 'w+') as f:
        f.writelines(molfile) #write molfile to temporary file
    mol = Chem.MolFromMolFile(temp_file) # Read temp file into RDKit molecule
    mol = Chem.AddHs(mol)# post-process molecule
    AllChem.EmbedMolecule(mol)
    Chem.MolToMolFile(mol, temp_file)    # Write RDKit molecule to a temporary file
    molecule = read('temp.mol') # Read the temporary file into ASE Atoms object
    return molecule



mol = molfile_to_xyz(content)

print(mol)


['\n', '     RDKit          3D\n', '\n', '  8  7  0  0  0  0  0  0  0  0999 V2000\n', '   -0.7467   -0.0178    0.0860 C   0  0  0  0  0  0  0  0  0  0  0  0\n', '    0.7705   -0.0461   -0.0328 C   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.0984    0.5678    0.9398 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.1004    0.5334   -0.8319 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '   -1.2007   -1.0086    0.0402 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.2712   -0.2380    0.9211 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.0495    1.0009   -0.3381 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '    1.0551   -0.7915   -0.7843 H   0  0  0  0  0  0  0  0  0  0  0  0\n', '  1  2  1  0\n', '  1  3  1  0\n', '  1  4  1  0\n', '  1  5  1  0\n', '  2  6  1  0\n', '  2  7  1  0\n', '  2  8  1  0\n', 'M  END\n']
Atoms(symbols='C2H6', pbc=False)


In [13]:
def smiles_to_xyz(smiles):
    mol = Chem.MolFromSmiles(smiles)
    print(mol)
    mol = Chem.AddHs(mol)
    print(mol)
    AllChem.EmbedMolecule(mol)
    print(mol)
    Chem.MolToMolFile(mol, 'temp.mol')    # Write RDKit molecule to a temporary file
    molecule = read('temp.mol') # Read the temporary file into ASE Atoms object
    return molecule

print(smiles_to_xyz('CC'))

{}
Atoms(symbols='C2H6', pbc=False)


In [3]:


def smiles_to_xyz(smiles):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    Chem.MolToMolFile(mol, 'temp.mol')    # Write RDKit molecule to a temporary file
    molecule = read('temp.mol') # Read the temporary file into ASE Atoms object
    return molecule